In [2]:
# --------------------------------------------
# STEP 1: Clone the official Shap-E repo
# --------------------------------------------
!git clone https://github.com/openai/shap-e.git
%cd shap-e

Cloning into 'shap-e'...
remote: Enumerating objects: 336, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 336 (delta 35), reused 13 (delta 13), pack-reused 281 (from 2)
Receiving objects: 100% (336/336), 11.72 MiB | 29.35 MiB/s, done.
Resolving deltas: 100% (43/43), done.
/content/shap-e


In [3]:
# --------------------------------------------
# STEP 2: Install Shap-E and required packages
# --------------------------------------------
!pip install -e .

Obtaining file:///content/shap-e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-z5xlga1j/clip_f31b757fe7e147ad9e4a30b1c44be18a
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-z5xlga1j/clip_f31b757fe7e147ad9e4a30b1c44be18a
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00


In [4]:
# --------------------------------------------
# STEP 3: Import all required libraries
# --------------------------------------------
import torch
# For sampling
from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config

# For loading pre-trained model
from shap_e.models.download import load_model, load_config

# For generating image preview and mesh export
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, decode_latent_mesh, gif_widget
from IPython.display import display
import time

In [5]:
# --------------------------------------------
# STEP 4: Set device to GPU (or CPU fallback)
# --------------------------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# --------------------------------------------
# STEP 5: Load models and diffusion configuration
# --------------------------------------------
# 'text300M' is the main text-to-3D model
model = load_model('text300M', device=device)

# Transmitter model for converting to mesh or renderable output
xm = load_model('transmitter', device=device)

# Load the diffusion settings
diffusion = diffusion_from_config(load_config('diffusion'))

/content/shap-e/shap_e/models/nn/checkpoint.py:31: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/content/shap-e/shap_e/models/nn/checkpoint.py:43: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/content/shap-e/shap_e/models/nn/checkpoint.py:61: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/content/shap-e/shap_e/models/nn/checkpoint.py:86: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
100%|███████████████████████████████████████| 890M/890M [00:09<00:00, 96.1MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

In [7]:
# --------------------------------------------
# STEP 6: Set your prompt here!
# --------------------------------------------
prompt = "a fantasy sword"  # 👈 Change this prompt to anything you want

In [8]:
# --------------------------------------------
# STEP 7: Generate 3D latent from text
# --------------------------------------------
# This part takes 1–2 minutes on GPU
start_time = time.time()

latents = sample_latents(
    batch_size=1,
    model=model,
    diffusion=diffusion,
    guidance_scale=15.0,
    model_kwargs=dict(texts=[prompt]),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,  # You can reduce to 32 if needed
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
    device=device,
)

end_time = time.time()
print(f"⏱️ Generation completed in {end_time - start_time:.2f} seconds")

  0%|          | 0/64 [00:00<?, ?it/s]

⏱️ Generation completed in 28.93 seconds


In [9]:
# --------------------------------------------
# STEP 8: Render a rotating preview (GIF)
# --------------------------------------------
cameras = create_pan_cameras(size=128, device=device)
images = decode_latent_images(xm, latents[0], cameras, rendering_mode='nerf')
display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAALW6XrixW7euWLqrV7KpVqPFZHjGZqi0WH6yZbuoVbinVLWnVr…

In [10]:
# --------------------------------------------
# STEP 9: Export the generated 3D mesh to OBJ
# --------------------------------------------
mesh = decode_latent_mesh(xm, latents[0])
with open('generated_model.obj', 'w') as f:
    mesh.tri_mesh().write_obj(f)

print("✅ 3D model exported as 'generated_model.obj'")

/content/shap-e/shap_e/models/stf/renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/content/shap-e/shap_e/models/stf/renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(


✅ 3D model exported as 'generated_model.obj'


In [12]:
# --------------------------------------------
# STEP 10: Optional — Export as PLY too (Fixed)
# --------------------------------------------
with open('generated_model.ply', 'wb') as f:
    mesh.tri_mesh().write_ply(f)

print("✅ 3D model also saved as 'generated_model.ply'")


✅ 3D model also saved as 'generated_model.ply'


In [13]:
%cd /content

# Move files to your cloned GitHub repo
!mv "Shap-E 3D Asset Generator.ipynb" 3d-asset-generator-shap-e/
!mv generated_model.obj generated_model.ply 3d-asset-generator-shap-e/


/content
mv: cannot stat 'Shap-E 3D Asset Generator.ipynb': No such file or directory
mv: target '3d-asset-generator-shap-e/' is not a directory


In [16]:
!git clone https://github.com/zainulabdin995/3d-asset-generator-shap-e.git


Cloning into '3d-asset-generator-shap-e'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [18]:
%%writefile /content/3d-asset-generator-shap-e/README.md
# 🧠 Shap-E 3D Asset Generator

This project demonstrates generating 3D models from text prompts using OpenAI’s Shap-E model.

---

## 🚀 Sample Prompt & Output

**Prompt:** `a fantasy sword`
**Output:** `generated_model.obj` + `generated_model.ply`

---

## 📦 Files

- `Shap-E 3D Asset Generator.ipynb` — main Colab notebook
- `generated_model.obj` / `.ply` — generated 3D mesh
- `README.md` — project overview

---

## 🛠️ How to Run

1. Open the notebook in Google Colab
2. Change the text prompt
3. Run all cells
4. Export `.obj` / `.ply` files


Overwriting /content/3d-asset-generator-shap-e/README.md


In [23]:
!ls /content/3d-asset-generator-shap-e/


generated_model.obj  generated_model.ply  README.md


In [27]:
!ls /content | grep '.ipynb'
